In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd

def load_and_print_ninapro_samples(file_path, num_samples=5):
    """
    NinaPro 데이터를 로드하고 처음 5개 샘플을 출력
    """
    print("=== NinaPro Data Sample Viewer ===")
    print(f"Loading data from: {file_path}\n")
    
    try:
        # .mat 파일 로드
        data = sio.loadmat(file_path)
        
        print("📁 Available keys in the data:")
        for key in data.keys():
            if not key.startswith('__'):
                print(f"  - {key}: {type(data[key])}, shape: {data[key].shape if hasattr(data[key], 'shape') else 'N/A'}")
        print()
        
        # EMG 데이터 추출
        if 'emg' in data:
            emg_data = data['emg']
            data_key = 'emg'
        elif 'data' in data:
            emg_data = data['data']
            data_key = 'data'
        else:
            data_keys = [k for k in data.keys() if not k.startswith('__')]
            emg_data = data[data_keys[0]]
            data_key = data_keys[0]
        
        # 라벨 데이터 추출
        if 'stimulus' in data:
            labels = data['stimulus'].flatten()
            label_key = 'stimulus'
        elif 'restimulus' in data:
            labels = data['restimulus'].flatten()
            label_key = 'restimulus'
        elif 'glove' in data:
            labels = data['glove']
            if labels.ndim > 1:
                labels = labels[:, 0]
            label_key = 'glove'
        else:
            label_keys = [k for k in data.keys() if 'stimulus' in k.lower() or 'label' in k.lower()]
            if label_keys:
                labels = data[label_keys[0]].flatten()
                label_key = label_keys[0]
            else:
                labels = np.arange(len(emg_data))  # 임시 라벨
                label_key = 'generated_labels'
        
        print(f"📊 Data Information:")
        print(f"  - EMG data key: '{data_key}'")
        print(f"  - EMG data shape: {emg_data.shape}")
        print(f"  - EMG data type: {emg_data.dtype}")
        print(f"  - Labels key: '{label_key}'")
        print(f"  - Labels shape: {labels.shape}")
        print(f"  - Unique labels: {np.unique(labels)}")
        print(f"  - Label counts: {np.bincount(labels.astype(int))}")
        print()
        
        # 5개 샘플 출력
        print(f"🔍 First {num_samples} samples:")
        print("-" * 80)
        
        for i in range(min(num_samples, len(emg_data))):
            print(f"Sample {i+1}:")
            print(f"  Label: {labels[i]}")
            print(f"  EMG channels (first 8): {emg_data[i][:8].round(4)}")
            if emg_data.shape[1] > 8:
                print(f"  ... (showing 8 out of {emg_data.shape[1]} channels)")
            print(f"  EMG min: {emg_data[i].min():.4f}, max: {emg_data[i].max():.4f}, mean: {emg_data[i].mean():.4f}")
            print()
        
        # 통계 정보
        print("📈 Overall Statistics:")
        print(f"  - Total samples: {len(emg_data)}")
        print(f"  - EMG channels: {emg_data.shape[1] if len(emg_data.shape) > 1 else 'N/A'}")
        print(f"  - EMG value range: [{emg_data.min():.4f}, {emg_data.max():.4f}]")
        print(f"  - EMG mean: {emg_data.mean():.4f}, std: {emg_data.std():.4f}")
        print()
        
        # 라벨별 분포
        unique_labels, counts = np.unique(labels, return_counts=True)
        print("🏷️  Label Distribution:")
        for label, count in zip(unique_labels, counts):
            percentage = (count / len(labels)) * 100
            print(f"  - Label {int(label)}: {count} samples ({percentage:.1f}%)")
        print()
        
        # 간단한 시각화 데이터 준비 (실제 plotting은 선택사항)
        print("📊 Ready for visualization...")
        print("  - Use matplotlib to plot EMG signals")
        print("  - First few samples can be plotted as time series")
        
        return emg_data, labels, data
        
    except FileNotFoundError:
        print(f"❌ Error: File not found at {file_path}")
        print("Generating sample data for demonstration...")
        return generate_sample_data(num_samples)
    
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        print("Generating sample data for demonstration...")
        return generate_sample_data(num_samples)

def generate_sample_data(num_samples=5):
    """
    샘플 데이터 생성 (파일이 없을 때)
    """
    print("🔧 Generating sample EMG-like data...")
    
    # 샘플 EMG 데이터 생성
    total_samples = 1000
    n_channels = 12
    emg_data = np.random.randn(total_samples, n_channels) * 0.1
    
    # EMG 신호처럼 보이도록 패턴 추가
    for i in range(n_channels):
        # 각 채널에 다른 주파수 성분 추가
        emg_data[:, i] += np.sin(np.linspace(0, 20*np.pi, total_samples) + i) * 0.05
        # 노이즈 추가
        emg_data[:, i] += np.random.randn(total_samples) * 0.02
    
    # 라벨 생성 (0: rest, 1-6: gestures)
    labels = np.random.choice([0, 1, 2, 3, 4, 5, 6], total_samples, 
                             p=[0.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])  # rest가 더 많음
    
    print(f"✅ Generated {total_samples} samples with {n_channels} channels")
    print(f"📊 Sample data overview:")
    for i in range(min(num_samples, total_samples)):
        print(f"  Sample {i+1}: Label={labels[i]}, EMG={emg_data[i][:4].round(3)}...")
    
    return emg_data, labels, {'emg': emg_data, 'stimulus': labels}

def plot_sample_signals(emg_data, labels, num_samples=5, num_channels=8):
    """
    샘플 신호들을 시각화
    """
    fig, axes = plt.subplots(num_samples, 1, figsize=(12, 2*num_samples))
    if num_samples == 1:
        axes = [axes]
    
    for i in range(min(num_samples, len(emg_data))):
        # 첫 번째 몇 개 채널만 플롯
        channels_to_plot = min(num_channels, emg_data.shape[1])
        
        for ch in range(channels_to_plot):
            axes[i].plot(emg_data[i, ch] if len(emg_data.shape) == 2 else [emg_data[i]], 
                        label=f'CH{ch+1}', alpha=0.7)
        
        axes[i].set_title(f'Sample {i+1} - Label: {labels[i]}')
        axes[i].set_ylabel('Amplitude')
        axes[i].grid(True, alpha=0.3)
        if i == 0 and channels_to_plot <= 8:
            axes[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    axes[-1].set_xlabel('Sample Index')
    plt.tight_layout()
    plt.show()

# 사용 예시
if __name__ == "__main__":
    # 데이터 파일 경로 설정
    DATA_PATH = "../../data/DB6/DB6_s1_a/S1_D1_T1.mat"  # 실제 경로로 변경
    
    # 또는 실제 파일 경로
    # DATA_PATH = "/users/acp24jhp/com6012/Dissertation/data/S1_D1_T1.mat"
    
    # 데이터 로드 및 출력
    emg_data, labels, raw_data = load_and_print_ninapro_samples(DATA_PATH, num_samples=5)
    
    # 선택적으로 시각화
    # plot_sample_signals(emg_data, labels, num_samples=5)
    
    print("✅ Data inspection complete!")

=== NinaPro Data Sample Viewer ===
Loading data from: ../../data/DB6/DB6_s1_a/S1_D1_T1.mat

📁 Available keys in the data:
  - repetition: <class 'numpy.ndarray'>, shape: (1428729, 1)
  - repetition_object: <class 'numpy.ndarray'>, shape: (1428729, 1)
  - acc: <class 'numpy.ndarray'>, shape: (1428729, 48)
  - daytesting: <class 'numpy.ndarray'>, shape: (1, 1)
  - emg: <class 'numpy.ndarray'>, shape: (1428729, 16)
  - object: <class 'numpy.ndarray'>, shape: (1428729, 1)
  - reobject: <class 'numpy.ndarray'>, shape: (1428729, 1)
  - rerepetition: <class 'numpy.ndarray'>, shape: (1428729, 1)
  - restimulus: <class 'numpy.ndarray'>, shape: (1428729, 1)
  - stimulus: <class 'numpy.ndarray'>, shape: (1428729, 1)
  - subj: <class 'numpy.ndarray'>, shape: (1, 1)
  - time: <class 'numpy.ndarray'>, shape: (1, 1)

📊 Data Information:
  - EMG data key: 'emg'
  - EMG data shape: (1428729, 16)
  - EMG data type: float32
  - Labels key: 'stimulus'
  - Labels shape: (1428729,)
  - Unique labels: [ 0  1

In [9]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

# ---------------------------
# 안전 접근 헬퍼
# ---------------------------
def safe_get(arr, i, default=0):
    """스칼라/길이1/길이N 모두에서 i번째 안전 접근"""
    if arr is None:
        return default
    arr = np.asarray(arr).squeeze()
    if arr.size == 0:
        return default
    if arr.ndim == 0:
        try:
            return arr.item()
        except Exception:
            return default
    idx = min(i, arr.shape[0]-1)
    v = arr[idx]
    try:
        return v.item()
    except Exception:
        return v

def safe_row(arr, i, width_if_none=0, fill=0.0):
    """(N, D) 배열에서 i번째 행을 안전 반환. 없으면 0 벡터."""
    if arr is None:
        return np.full(width_if_none, fill, dtype=float)
    a = np.asarray(arr)
    if a.ndim == 1:  # (N,)이면 피처가 1개라고 가정
        a = a[:, None]
    if a.shape[0] == 0:
        return np.full(a.shape[1], fill, dtype=float)
    idx = min(i, a.shape[0]-1)
    return a[idx]

def load_and_display_ninapro_table(file_path, num_samples=5):
    print("=" * 80)
    print("🔬 NinaPro Dataset Table Viewer")
    print("=" * 80)
    print(f"📁 Loading data from: {file_path}\n")
    
    try:
        data = sio.loadmat(file_path)

        print("📋 Available Data Keys:")
        print("-" * 40)
        for key in sorted(k for k in data.keys() if not k.startswith('__')):
            shape_info = f"({data[key].shape})" if hasattr(data[key], 'shape') else ""
            print(f"  ✓ {key:<15} {shape_info}")
        print()
        
        extracted_data = extract_ninapro_data(data)
        print_data_summary(extracted_data)
        display_sample_table(extracted_data, num_samples)
        display_emg_channel_stats(extracted_data, num_samples)
        display_label_distribution(extracted_data)
        return extracted_data
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        print("🔧 Generating sample NinaPro-like data...\n")
        return generate_sample_ninapro_data(num_samples)
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        print("🔧 Generating sample NinaPro-like data...\n")
        return generate_sample_ninapro_data(num_samples)

def extract_ninapro_data(data):
    extracted = {}
    extracted['emg'] = data.get('emg', None)
    extracted['acc'] = data.get('acc', None)
    for k in ['stimulus','restimulus','object','reobject',
              'repetition','rerepetition','repetition_object',
              'daytesting','time']:
        v = data.get(k, np.array([]))
        try:
            extracted[k] = np.asarray(v).flatten()
        except Exception:
            extracted[k] = np.array([])
    return extracted

def print_data_summary(data):
    print("📊 Dataset Summary")
    print("-" * 50)
    emg_shape = data['emg'].shape if data['emg'] is not None else (0, 0)
    acc_shape = data['acc'].shape if data['acc'] is not None else (0, 0)

    # 유니크 계산 시 빈 배열 보호
    stim = data['stimulus'] if data['stimulus'].size else np.array([])
    obj  = data['object'] if data['object'].size else np.array([])
    day  = data['daytesting'] if data['daytesting'].size else np.array([])
    tim  = data['time'] if data['time'].size else np.array([])

    summary_data = [
        ["Total Samples", f"{emg_shape[0]:,}"],
        ["EMG Channels", f"{emg_shape[1]} (14 active + 2 empty)"],
        ["ACC Channels", f"{acc_shape[1]} (12 electrodes × 3 axes)"],
        ["Unique Gestures", f"{len(np.unique(stim[stim > 0])) if stim.size else 0}"],
        ["Unique Objects", f"{len(np.unique(obj[obj > 0])) if obj.size else 0}"],
        ["Collection Days", f"{len(np.unique(day[day > 0])) if day.size else 0}"],
        ["Time Sessions", f"{len(np.unique(tim[tim > 0])) if tim.size else 0}"]
    ]
    print(tabulate(summary_data, headers=["Metric", "Value"], tablefmt="grid"))
    print()

def display_sample_table(data, num_samples=5):
    print(f"📋 Sample Data Table (First {num_samples} rows)")
    print("=" * 120)
    table_data = []

    total = min(num_samples, max(len(data['stimulus']), 1))
    for i in range(total):
        # EMG 첫 4채널
        emg_row = safe_row(data['emg'], i, width_if_none=4)
        emg_sample = emg_row[:4]
        emg_str = f"[{', '.join([f'{x:.3f}' for x in emg_sample])}...]"

        # ACC 첫 3값
        acc_row = safe_row(data['acc'], i, width_if_none=3)
        acc_sample = acc_row[:3]
        acc_str = f"[{', '.join([f'{x:.3f}' for x in acc_sample])}...]"

        # 시간 정보
        t = safe_get(data.get('time'), i, default=0)
        time_session = "Morning" if t == 1 else ("Afternoon" if t == 2 else "Unknown")

        row = [
            i + 1,
            int(safe_get(data.get('stimulus'), i, default=0)),
            int(safe_get(data.get('restimulus'), i, default=0)),
            int(safe_get(data.get('object'), i, default=0)),
            int(safe_get(data.get('repetition'), i, default=0)),
            int(safe_get(data.get('daytesting'), i, default=1)),
            time_session,
            emg_str,
            acc_str
        ]
        table_data.append(row)

    headers = ["Index", "Gesture", "Refined\nGesture", "Object", "Rep", "Day", "Time",
               "EMG (first 4 ch)", "ACC (first 3 val)"]
    print(tabulate(table_data, headers=headers, tablefmt="grid", stralign="center"))
    print()

def display_emg_channel_stats(data, num_samples=5):
    if data['emg'] is None or data['emg'].size == 0:
        return
    print(f"⚡ EMG Channel Statistics (First {num_samples} samples)")
    print("-" * 80)

    stats_data = []
    total = min(num_samples, data['emg'].shape[0])
    for i in range(total):
        sample_emg = safe_row(data['emg'], i)
        active_channels = sample_emg[:14]
        stats_row = [
            f"Sample {i+1}",
            f"{active_channels.min():.4f}",
            f"{active_channels.max():.4f}",
            f"{active_channels.mean():.4f}",
            f"{active_channels.std():.4f}",
            f"{np.sum(np.abs(active_channels) > 0.01)}",
            f"Gesture {int(safe_get(data.get('stimulus'), i, default=0))}"
        ]
        stats_data.append(stats_row)

    headers = ["Sample", "Min", "Max", "Mean", "Std", "Active Ch", "Label"]
    print(tabulate(stats_data, headers=headers, tablefmt="grid", floatfmt=".4f"))
    print()

def display_label_distribution(data):
    print("🏷️  Gesture Label Distribution")
    print("-" * 50)

    stim = data['stimulus'] if data['stimulus'].size else np.array([])
    if stim.size == 0:
        print("(no labels)\n")
        return

    unique_labels, counts = np.unique(stim, return_counts=True)
    gesture_names = {
        0: "Rest",
        1: "Index flexion", 2: "Index extension", 3: "Middle flexion",
        4: "Middle extension", 5: "Ring flexion", 6: "Ring extension",
        7: "Little flexion", 8: "Little extension", 9: "Thumb up",
        10: "Thumb down", 11: "Thumb left", 12: "Thumb right"
    }

    dist_data = []
    total_samples = len(stim)
    for label, count in zip(unique_labels, counts):
        percentage = (count / total_samples * 100.0) if total_samples > 0 else 0.0
        gesture_name = gesture_names.get(int(label), f"Gesture {int(label)}")
        dist_data.append([int(label), gesture_name, f"{count:,}", f"{percentage:.1f}%"])

    headers = ["Label", "Gesture Type", "Count", "Percentage"]
    print(tabulate(dist_data, headers=headers, tablefmt="grid", stralign="left"))
    print()

    # Object 분포
    obj = data['object'] if data['object'].size else np.array([])
    if obj.size and np.any(obj > 0):
        print("📦 Object Label Distribution")
        print("-" * 30)
        unique_objects, obj_counts = np.unique(obj[obj > 0], return_counts=True)
        denom = len(obj[obj > 0])
        obj_data = []
        for o, c in zip(unique_objects, obj_counts):
            percentage = (c / denom * 100.0) if denom > 0 else 0.0
            obj_data.append([int(o), f"{c:,}", f"{percentage:.1f}%"])
        print(tabulate(obj_data, headers=["Object ID", "Count", "Percentage"],
                       tablefmt="grid", stralign="center"))
        print()

def generate_sample_ninapro_data(num_samples=5):
    print("🔧 Generating Sample NinaPro Data...")
    total_samples = max(100, num_samples * 10)

    emg_data = np.random.randn(total_samples, 16) * 0.05
    emg_data[:, 14:] = 0  # 마지막 2채널 비움
    acc_data = np.random.randn(total_samples, 48) * 0.1

    # ✅ 확률 합 1.0로 수정
    rest_prob = 0.30
    n_gestures = 12  # 1~12
    p = [rest_prob] + [ (1.0 - rest_prob) / n_gestures ] * n_gestures  # 길이 13
    stimulus = np.random.choice(np.arange(0, n_gestures+1), size=total_samples, p=p)
    restimulus = stimulus.copy()

    object_labels = np.random.choice([0, 1, 2, 3, 4, 5], total_samples,
                                     p=[0.5, 0.1, 0.1, 0.1, 0.1, 0.1])
    repetition = np.random.randint(1, 7, total_samples)
    daytesting = np.random.randint(1, 6, total_samples)
    time_session = np.random.choice([1, 2], total_samples)  # 1=morning, 2=afternoon

    return {
        'emg': emg_data,
        'acc': acc_data,
        'stimulus': stimulus,
        'restimulus': restimulus,
        'object': object_labels,
        'reobject': object_labels,
        'repetition': repetition,
        'rerepetition': repetition,
        'repetition_object': repetition,
        'daytesting': daytesting,
        'time': time_session
    }

def export_to_csv(data, num_samples=10, filename="ninapro_sample.csv"):
    print(f"💾 Exporting {num_samples} samples to {filename}...")
    export_data = []

    total = min(num_samples, max(len(data['stimulus']), 1))
    for i in range(total):
        t = safe_get(data.get('time'), i, default=0)
        row = {
            'Index': i + 1,
            'Gesture': int(safe_get(data.get('stimulus'), i, default=0)),
            'Refined_Gesture': int(safe_get(data.get('restimulus'), i, default=0)),
            'Object': int(safe_get(data.get('object'), i, default=0)),
            'Repetition': int(safe_get(data.get('repetition'), i, default=0)),
            'Day': int(safe_get(data.get('daytesting'), i, default=1)),
            'Time_Session': 'Morning' if t == 1 else ('Afternoon' if t == 2 else 'Unknown')
        }

        emg_row = safe_row(data.get('emg'), i)
        for ch in range(min(8, emg_row.shape[0] if emg_row.ndim == 1 else emg_row.shape[-1])):
            row[f'EMG_CH{ch+1}'] = float(emg_row[ch]) if emg_row.ndim == 1 else float(emg_row[..., ch])

        acc_row = safe_row(data.get('acc'), i)
        for k in range(min(6, acc_row.shape[0] if acc_row.ndim == 1 else acc_row.shape[-1])):
            row[f'ACC_{k+1}'] = float(acc_row[k]) if acc_row.ndim == 1 else float(acc_row[..., k])

        export_data.append(row)

    df = pd.DataFrame(export_data)
    df.to_csv(filename, index=False)
    print(f"✅ Data exported to {filename}")
    return df
def display_sample_table(data, num_samples=5, random=False, seed=None):
    """
    샘플 데이터를 Excel 표 형식으로 출력
    - random=True 이면 전체에서 무작위로 num_samples개 추출
    """
    print(f"📋 Sample Data Table ({'Random ' if random else 'First '}{num_samples} rows)")
    print("=" * 120)

    # 전체 길이 계산(가용한 키 중 최대 길이)
    n = max(
        len(data.get('stimulus', [])),
        data['emg'].shape[0] if data.get('emg') is not None else 0,
        data['acc'].shape[0] if data.get('acc') is not None else 0
    )
    if n == 0:
        print("(no data)\n"); return

    k = min(num_samples, n)

    # 인덱스 선택
    if random:
        rng = np.random.default_rng(seed)
        indices = rng.choice(n, size=k, replace=False)
    else:
        indices = np.arange(k)

    table_data = []
    for i in indices:
        # EMG 첫 4채널
        emg_row = safe_row(data.get('emg'), i, width_if_none=4)
        emg_sample = emg_row[:4]
        emg_str = f"[{', '.join([f'{x:.3f}' for x in emg_sample])}...]"

        # ACC 첫 3값
        acc_row = safe_row(data.get('acc'), i, width_if_none=3)
        acc_sample = acc_row[:3]
        acc_str = f"[{', '.join([f'{x:.3f}' for x in acc_sample])}...]"

        # 시간 세션
        t = safe_get(data.get('time'), i, default=0)
        time_session = "Morning" if t == 1 else ("Afternoon" if t == 2 else "Unknown")

        row = [
            i + 1,  # 원본 인덱스 표시
            int(safe_get(data.get('stimulus'), i, default=0)),
            int(safe_get(data.get('restimulus'), i, default=0)),
            int(safe_get(data.get('object'), i, default=0)),
            int(safe_get(data.get('repetition'), i, default=0)),
            int(safe_get(data.get('daytesting'), i, default=1)),
            time_session,
            emg_str,
            acc_str
        ]
        table_data.append(row)

    headers = [
        "Index", "Gesture", "Refined\nGesture", "Object", "Rep", "Day", "Time",
        "EMG (first 4 ch)", "ACC (first 3 val)"
    ]
    print(tabulate(table_data, headers=headers, tablefmt="grid", stralign="center"))
    print()

# 사용 예시
if __name__ == "__main__":
    DATA_PATH =  "../../data/DB6/DB6_s1_a/S1_D1_T1.mat"
    print("🚀 Starting NinaPro Data Analysis...\n")
    display_sample_table(ninapro_data, num_samples=10, random=True, seed=42)
    print("✅ Analysis Complete!")
    print("-" * 80)
    csv_df = export_to_csv(ninapro_data, num_samples=10)


🚀 Starting NinaPro Data Analysis...

📋 Sample Data Table (Random 10 rows)
+------------------+-----------+-----------+----------+-------+-------+---------+-------------------------------------+----------------------------+
|            Index |   Gesture |   Refined |   Object |   Rep |   Day |  Time   |          EMG (first 4 ch)           |     ACC (first 3 val)      |
|                  |           |   Gesture |          |       |       |         |                                     |                            |
+==================+===========+===========+==========+=======+=======+=========+=====================================+============================+
| 122793           |         1 |         1 |        4 |     8 |     1 | Morning |   [0.000, 0.000, 0.000, 0.000...]   | [0.081, 1.034, -0.418...]  |
+------------------+-----------+-----------+----------+-------+-------+---------+-------------------------------------+----------------------------+
|      1.10577e+06 |         0 |